In [3]:
from keras_segmentation.models.fcn import fcn_8_vgg

# from keras_segmentation.models.unet import vgg_unet
from keras.callbacks import Callback, ModelCheckpoint

import os
import time
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import keras
import numpy as np
import glob
import cv2

In [4]:
class TrainingPlot(Callback):
    # This function is called when the training begins
    def on_train_begin(self, logs=None):
        # Initialize the lists for holding the logs, losses and accuracies
        self.losses = []
        self.acc = []
        self.val_losses = []
        self.val_acc = []
        self.logs = []

    def on_train_end(self, logs=None):
        # only in train end, draw log figure
        # Before plotting ensure at least 2 epochs have passed
        if len(self.losses) > 1:
            N = np.arange(0, len(self.losses))

            # You can chose the style of your preference
            # print(plt.style.available) to see the available options
            #plt.style.use("seaborn")
            colors=['orange', 'purple', 'green','red']
            plt.gca().set_prop_cycle(color=colors)
            
            # Plot train loss, train acc, val loss and val acc against epochs passed
            plt.figure()
            plt.ylim(0, 1) 
            plt.plot(N, self.losses, label = "train_loss")
            plt.plot(N, self.acc, label = "train_accuracy")
            plt.plot(N, self.val_losses, label = "val_loss")
            plt.plot(N, self.val_acc, label = "val_accuracy")
            #plt.title("Training Loss and Accuracy [Epoch {}]".format(epoch))
            plt.title("Training Loss and Accuracy")

            plt.xlabel("Epoch")
            plt.ylabel("Loss/Accuracy")
            plt.legend()
            # Make sure there exists a folder called output in the current directory
            # or replace 'output' with whatever direcory you want to put in the plots
            plt.savefig('000_log.png')
            plt.show()
            plt.close()

    # This function is called at the end of each epoch
    def on_epoch_end(self, epoch, logs=None):
        # print('logs:',logs)

        # Append the logs, losses and accuracies to the lists
        self.logs.append(logs)
        
        a = logs.get('loss')
        b = logs.get('val_loss')
        if a > 1:
            a = 0.9999
        if b > 1:
            b = 0.9999
        self.losses.append(logs.get('loss'))
        self.acc.append(logs.get('accuracy'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_accuracy'))

In [5]:
trainplot = TrainingPlot()

In [6]:
# start processing......    
model = fcn_8_vgg(n_classes=11, input_height=416, input_width=608)
cbs = [ModelCheckpoint("000_weight.h5", save_best_only=True), 
       trainplot,]

In [ ]:
model.train(
    train_images =  "/kaggle/input/football-seg-v3/train_img/",
    train_annotations = "/kaggle/input/football-seg-v3/train_label/",
    #checkpoints_path = "/kaggle/input/football-seg-v3/checkpoints/pspnet_101_finetune_dataset2/pspnet_101_finetune_dataset2" ,
    epochs=50,
    validate=True,
    val_images='/kaggle/input/football-seg-v3/val_img/',
    val_annotations='/kaggle/input/football-seg-v3/val_label/',
    batch_size = 2,
    steps_per_epoch = 40, # 40
    val_steps_per_epoch = 40, # 40
    callbacks = cbs,
)

Verifying training dataset


100%|██████████| 83/83 [00:06<00:00, 13.59it/s]


Dataset verified! 
Verifying validation dataset


100%|██████████| 17/17 [00:01<00:00, 14.40it/s]


Dataset verified! 
Epoch 1/50
40/40 [==============================] - 25s 556ms/step - loss: 15.1293 - accuracy: 0.1722 - val_loss: 2.0466 - val_accuracy: 0.2353


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
2022-12-27 01:38:24.860351: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 411041792 exceeds 10% of free system memory.
2022-12-27 01:38:26.438184: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 411041792 exceeds 10% of free system memory.
2022-12-27 01:38:27.347763: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 411041792 exceeds 10% of free system memory.


Epoch 2/50
40/40 [==============================] - 22s 556ms/step - loss: 1.9517 - accuracy: 0.2473 - val_loss: 1.8244 - val_accuracy: 0.2721


2022-12-27 01:38:55.368925: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 411041792 exceeds 10% of free system memory.
2022-12-27 01:38:56.360621: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 411041792 exceeds 10% of free system memory.


Epoch 3/50
40/40 [==============================] - 22s 551ms/step - loss: 1.7303 - accuracy: 0.3254 - val_loss: 1.5071 - val_accuracy: 0.4432
Epoch 4/50
40/40 [==============================] - 21s 542ms/step - loss: 1.6269 - accuracy: 0.4317 - val_loss: 1.4412 - val_accuracy: 0.4608
Epoch 5/50
40/40 [==============================] - 21s 545ms/step - loss: 1.3101 - accuracy: 0.5110 - val_loss: 1.1293 - val_accuracy: 0.5804
Epoch 6/50
40/40 [==============================] - 22s 547ms/step - loss: 1.0949 - accuracy: 0.6328 - val_loss: 0.9369 - val_accuracy: 0.7237
Epoch 7/50
40/40 [==============================] - 22s 560ms/step - loss: 0.9380 - accuracy: 0.6994 - val_loss: 0.8068 - val_accuracy: 0.7627
Epoch 8/50
40/40 [==============================] - 22s 550ms/step - loss: 0.8105 - accuracy: 0.7446 - val_loss: 0.6996 - val_accuracy: 0.7818
Epoch 9/50
40/40 [==============================] - 21s 538ms/step - loss: 0.7567 - accuracy: 0.7541 - val_loss: 0.6791 - val_accuracy: 0.7838

In [ ]:
def show_multi_imgs(scale, imglist, order=None, border=10, border_color=(255, 255, 0)):
    """
    :param scale: float 原图缩放的尺度
    :param imglist: list 待显示的图像序列
    :param order: list or tuple 显示顺序 行×列
    :param border: int 图像间隔距离
    :param border_color: tuple 间隔区域颜色
    :return: 返回拼接好的numpy数组
    """
    if order is None:
        order = [1, len(imglist)]
    allimgs = imglist.copy()
    ws , hs = [], []
    for i, img in enumerate(allimgs):
        if np.ndim(img) == 2:
            allimgs[i] = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        allimgs[i] = cv2.resize(img, dsize=(0, 0), fx=scale, fy=scale)
        ws.append(allimgs[i].shape[1])
        hs.append(allimgs[i].shape[0])
    w = max(ws)
    h = max(hs)
    # 将待显示图片拼接起来
    sub = int(order[0] * order[1] - len(imglist))
    # 判断输入的显示格式与待显示图像数量的大小关系
    if sub > 0:
        for s in range(sub):
            allimgs.append(np.zeros_like(allimgs[0]))
    elif sub < 0:
        allimgs = allimgs[:sub]
    imgblank = np.zeros(((h+border) * order[0], (w+border) * order[1], 3)) + border_color
    imgblank = imgblank.astype(np.uint8)
    for i in range(order[0]):
        for j in range(order[1]):
            imgblank[(i * h + i*border):((i + 1) * h+i*border), (j * w + j*border):((j + 1) * w + j*border), :] = allimgs[i * order[1] + j]
    return imgblank

In [ ]:
val_img_path = '/kaggle/input/football-seg-v3/val_img/*'
val_img_list = glob.glob(val_img_path)
val_img_list.sort()

val_result = []
for i in range(len(val_img_list)):
    out = model.predict_maps( inp=val_img_list[i] )
    # print('out.shape',out.shape)
    # cv2.imshow(str(i), out)
    # cv2.waitKey(0)
#     out = out[:, :, (2, 1, 0)] # cv2的BGR->plt的RGB
    val_result.append(out)

In [ ]:
# show_and_save_maps(val_result)
img = show_multi_imgs(0.5, val_result, (6,3))
plt.imshow(img[:, :, (2, 1, 0)])
#cv2.namedWindow('val_result_multi', 0)
#cv2.imshow('val_result_multi', img)

cv2.imwrite('000_val_result_multi.png', img)

In [ ]:
# evaluate IoU fIoU...
eval_result = model.evaluate_segmentation( inp_images_dir="/kaggle/input/football-seg-v3/val_img/"  , 
    annotations_dir="/kaggle/input/football-seg-v3/val_label/" ) 

In [ ]:
def number2string_format(inp):
    # 保留4位小数点（四舍五入）：round(l[i], 4)
    # 小数点未满4位的填补0："{:.4f}".format()
    return str("{:.4f}".format(round(inp, 4)))

def list2string(lis):
    # 输入一个list，输出一行string，list的每个数字保留4位小数，数字之间用空格隔开
    s = ''
    for i in range(len(lis)):
        s += number2string_format(lis[i])
        s += ' '
    return s

def write2txt(loss, eval_result, filename):
    file = open(filename, "w")
    i = 0

    # 先把eval_result写入
    for value in eval_result.values():
        if i < 2 : # frequency_weighted_IU, mean_IU
            value_format = number2string_format(value)
        else: # class_wise_IU, class_wise_pixels_norm
            value_format= list2string(value)
        file.write(value_format)
        file.write('\n')
        i += 1

    # 再把loss写入
    for value in loss.values():
        value_format= list2string(value)
        file.write(value_format)
        file.write('\n')

    file.close()

In [ ]:
loss_dict = {'train_losses': trainplot.losses,
            'train_acc': trainplot.acc,
            'val_losses': trainplot.val_losses,
            'val_acc':trainplot.val_acc}

# save to txt
write2txt(loss_dict,eval_result, "000_eval.txt")

In [ ]:
print('finish')